# Iteration 3

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nuim68ph/unsloth_7b93e6b651ff48be880a1e34fccf6e55
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nuim68ph/unsloth_7b93e6b651ff48be880a1e34fccf6e55
  Resolved https://github.com/unslothai/unsloth.git to commit ec1757c1a02175851146ff5f6ab2a26c8c863fc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.8/180.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 149.4 MB/s eta 0:00:00

In [2]:
from unsloth import FastLanguageModel
import torch
import json
import re
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
MODEL_ID = "unsloth/Qwen2.5-32B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 4096
NEW_LR = 5e-5

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = True,
)

⏳ Loading Unsloth Model...
==((====))==  Unsloth 2026.1.2: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2026.1.2 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


In [ ]:
def format_data(examples):
    texts = []
    for question, answers, correct in zip(examples['question'], examples['answers'], examples['correct_answers']):
        options = "\n".join([f"{ans['marker']}) {ans['text']}" for ans in answers])

        sys_msg = "Ви — екзаменатор. Визначте правильну відповідь на запитання ЗНО. Виведіть ТІЛЬКИ букву."

        user_msg = f"{question}\n\nВаріанти:\n{options}\nВкажіть правильну літеру:"
        assistant_msg = correct[0] # The correct letter (e.g., "А")

        conversation = [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]

        text = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    return {"text": texts}

In [ ]:
print("Loading and formatting data...")
data = []
with open("zno.train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

Loading and formatting data...


In [15]:
raw_dataset = Dataset.from_list(data)
dataset = raw_dataset.map(format_data, batched=True)

Map:   0%|          | 0/3063 [00:00<?, ? examples/s]

In [ ]:
print(" Starting Training (SFT)...")
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = NEW_LR,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        output_dir = "outputs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
    ),
)

🚀 Starting Training (SFT)...


Map (num_proc=2):   0%|          | 0/3063 [00:00<?, ? examples/s]

In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,063 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 134,217,728 of 32,898,094,080 (0.41% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.337600
20,1.792000
30,1.482100
40,1.382500
50,1.387900
60,1.264100


TrainOutput(global_step=60, training_loss=1.6077135721842448, metrics={'train_runtime': 409.8458, 'train_samples_per_second': 1.171, 'train_steps_per_second': 0.146, 'total_flos': 5.10704324103168e+16, 'train_loss': 1.6077135721842448, 'epoch': 0.1566579634464752})

In [ ]:
print("Saving Adapter...")
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print("Done! You now have a fine-tuned ZNO expert.")

💾 Saving Adapter...
✅ Done! You now have a fine-tuned ZNO expert.


In [19]:
ADAPTER_MODEL = "lora_model"
INPUT_FILE = "zno.test.jsonl"
OUTPUT_FILE = "submission_sft.csv"

In [ ]:
print(f"Loading Fine-Tuned Model from {ADAPTER_MODEL}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = ADAPTER_MODEL,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

⏳ Loading Fine-Tuned Model from lora_model...
==((====))==  Unsloth 2026.1.2: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120, padding_idx=151665)
        (layers): ModuleList(
          (0-63): 64 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [21]:
print("Loading test data...")
data = []
import json
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

Loading test data...


In [ ]:
def format_prompt_inference(item):
    question = item['question']
    options_text = ""
    valid_markers = []
    for ans in item['answers']:
        options_text += f"{ans['marker']}) {ans['text']}\n"
        valid_markers.append(ans['marker'])

    messages = [
        {"role": "system", "content": "Ви — екзаменатор. Визначте правильну відповідь на запитання ЗНО. Виведіть ТІЛЬКИ букву."},
        {"role": "user", "content": f"{question}\n\nВаріанти:\n{options_text}\nВкажіть правильну літеру:"}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True
    )
    return text, valid_markers

In [ ]:
results = []
print(f"\nStarting Evaluation on {len(data)} items...")

for i, item in enumerate(tqdm(data)):
    prompt, valid_markers = format_prompt_inference(item)
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 2,   
            temperature = 0.1,    
            do_sample = False,
            use_cache = True,
        )

    response = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

    cleaned_response = response.strip().replace(".", "")

    pred = "А"
    if cleaned_response in valid_markers:
        pred = cleaned_response
    else:
        match = re.search(r'([АБВГД])', response)
        if match:
            pred = match.group(1)

    results.append({"id": item['id'], "correct_answers": pred})

    if i < 10:
        print(f"Question: {item['question'][:40]}... -> Pred: {pred} (Raw: {response})")


🚀 Starting Evaluation on 751 items...


  0%|          | 1/751 [00:01<16:58,  1.36s/it]

Question: «Сміхом крізь сльози» можна схарактеризу... -> Pred: Б (Raw: Б)


  0%|          | 2/751 [00:01<10:15,  1.22it/s]

Question: Удовин син, мати, сестра, кохана – ключо... -> Pred: В (Raw: В)


...

  1%|          | 9/751 [00:04<05:43,  2.16it/s]

Question: У ході «Чортківської офензиви» Українськ... -> Pred: В (Raw: В)


  1%|▏         | 10/751 [00:05<05:58,  2.07it/s]

Question: Що стримувало «радянізацію» Західної Укр... -> Pred: А (Raw: А)


100%|██████████| 751/751 [05:45<00:00,  2.17it/s]


In [ ]:
df = pd.DataFrame(results)
df.to_csv(OUTPUT_FILE, index=False)
print(f"SFT Submission Saved: {OUTPUT_FILE}")

✅ SFT Submission Saved: submission_sft.csv
